# Advanced Features (Creativity Layer)

This notebook demonstrates advanced behavior of our RAG-based book recommender:

- Mood-aware tone adaptation (happy/sad/neutral)
- Explain-why mode that explicitly justifies each recommendation
- Second-opinion mode that provides an alternative set of suggestions

These features build on top of the base RAG pipeline and show how we further
refine the generator's behavior beyond a basic LLM setup.


In [1]:
import os
import sys

project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

from src.retriever.retriever import Retriever
from src.generator.rag_generator import RAGGenerator
from src.pipeline.rag_pipeline import RAGPipeline

/Users/biancaleoveanu/Desktop/rag-book-recommender/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
retriever = Retriever(
    index_path=os.path.join(project_root, "models", "faiss_index.bin"),
    metadata_path=os.path.join(project_root, "models", "metadata.pkl"),
    model_name="all-MiniLM-L6-v2",
    k=6,
)

gen = RAGGenerator(model="llama3")
pipeline = RAGPipeline(retriever=retriever, generator=gen, k=6)

Loading embedding model: all-MiniLM-L6-v2 on cpu
Loaded FAISS index from /Users/biancaleoveanu/Desktop/rag-book-recommender/models/faiss_index.bin
Loaded metadata from /Users/biancaleoveanu/Desktop/rag-book-recommender/models/metadata.pkl (n=10710)


In [3]:
from src.utils.mood import detect_mood

query_happy = "I'm feeling great today and want a fun, light fantasy romance."
query_sad = "I'm feeling a bit down and want something comforting and hopeful."

print("Detected mood (happy query):", detect_mood(query_happy))
print("Detected mood (sad query):", detect_mood(query_sad))

out_happy = pipeline.run(query_happy, style="detailed", personality="friendly", explain=True)
out_sad = pipeline.run(query_sad, style="detailed", personality="friendly", explain=True)

print("\n=== HAPPY QUERY ANSWER ===\n")
print(out_happy["answer"])

print("\n=== SAD QUERY ANSWER ===\n")
print(out_sad["answer"])

Device set to use mps:0


Detected mood (happy query): happy
Detected mood (sad query): sad


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== HAPPY QUERY ANSWER ===

I'm thrilled to recommend some fantastic options for you!

Based on your query for a fun, light fantasy romance, I highly recommend the following books:

* **A Whole New Light** by Sandra Brown — This book is a great match as it's a lighthearted romance with a touch of humor. The rating suggests it's an enjoyable read, and the genres listed include romance, which aligns well with your query.
* **Embracing Love Vol. 4** by Youka Nitta — As a yaoi manga series, this book combines fantasy elements with romance, making it a great fit for your request. The high rating (4.34) indicates that readers have thoroughly enjoyed the series.

While there aren't many books on the list that directly match fantasy romance, I can suggest an alternative that might still be of interest:

* **Fantasy Gone Wrong** by Martin H. Greenberg/Brittiany A. Koren/Brian Stableford/Mickey Zucker Reichert/Fiona Patton/Jim C. Hines/Esther M. Friesner/Donald J. Bingle/Alan Dean Foster/Devon 

In [4]:
q = "Historic fiction with a strong female lead."

out_default = pipeline.run(q, style="detailed", personality="academic", explain=False)
out_explain = pipeline.run(q, style="detailed", personality="academic", explain=True)

print("=== WITHOUT explain=True ===\n")
print(out_default["answer"])

print("\n=== WITH explain=True ===\n")
print(out_explain["answer"])

=== WITHOUT explain=True ===

Based on your query for historic fiction with a strong female lead, I recommend the following two books:

• **The Woman in White** by Wilkie Collins — This classic novel features a strong female lead, Marian Halcombe, who plays a crucial role in uncovering the mystery surrounding Laura Fairlie's disappearance. The book is set in the 19th century and combines elements of gothic horror, mystery, and romance.

• **The Women's War** by Alexandre Dumas (translated by Robin Buss) — Although not exclusively focused on a single strong female lead, this historical novel features a cast of powerful women who shape the events of the story. Set in 18th-century France, the book explores themes of love, war, and social change.

As these two books meet your query criteria, I did not find any other matches from the retrieved list that better fit your request. However, if you're interested in exploring more options, **A Woman in Jerusalem** by A.B. Yehoshua (translated by 

In [5]:
q = "Epic fantasy with complex worldbuilding and politics."

history = []

first = pipeline.run(q, style="detailed", personality="friendly", explain=True, history=history)
history.append({"user": q, "bot": first["answer"]})

second = pipeline.run(
    q,
    style="detailed",
    personality="friendly",
    explain=True,
    second_opinion=True,
    history=history,
)

print("=== FIRST RECOMMENDATION ===\n")
print(first["answer"])

print("\n=== SECOND OPINION ===\n")
print(second["answer"])

=== FIRST RECOMMENDATION ===

I'm so excited to recommend some epic fantasy books for you!

Based on your query, I highly recommend the following books:

* **Into a Dark Realm** by Raymond E. Feist — This book is part of The Darkwar Saga series, which is known for its complex worldbuilding and politics. As an epic fantasy novel, it's sure to satisfy your craving for intricate storytelling and adventure.

* **Road of the Patriarch** by R.A. Salvatore — Another epic fantasy novel on this list, Road of the Patriarch is part of The Sellswords series and features complex worldbuilding, politics, and high-stakes action. As a fan of Forgotten Realms, you'll likely enjoy this book's immersive setting and characters.

* **The Magical Worlds of Lord of the Rings** by David Colbert — While not an epic fantasy novel itself, this book is a fascinating exploration of the mythology and lore behind J.R.R. Tolkien's masterpiece. If you're interested in the worldbuilding and politics of Middle-earth, th

In this notebook we extended the basic RAG pipeline with several generator-side features:

- **Mood-aware recommendations:**  
  We detect a simple mood label (`happy`, `sad`, `neutral`) from the user query using a HuggingFace sentiment model plus a small keyword heuristic.  
  The detected mood is injected into the system prompt so the LLM adapts its tone.  
  For example, a happy query leads to fun, light fantasy romance suggestions, while a sad query triggers comforting, hopeful self-help style recommendations.

- **Explain mode:**  
  The `explain=True` flag changes the task instruction so the LLM explicitly justifies each recommendation (genres, themes, ratings, year) and may also mention why some retrieved books were not chosen.  
  We showed side-by-side outputs for the same query with and without `explain=True`.

- **Second opinion mode:**  
  The `second_opinion=True` flag asks the LLM to provide an alternative perspective to a previous answer using the same retrieved books and chat history.  
  In our tests, the second opinion mostly rephrased and slightly re-weighted the recommendations, which we discuss as a possible area for further tuning.

  Additionally, the second-opinion mode was enhanced with a higher temperature and explicit instructions encouraging variation, so the model provides an alternative perspective rather than repeating the previous recommendation verbatim.

These features demonstrate that we do not just call a base LLM, but **shape its behaviour** via prompt design, mood detection, and additional control flags on top of the RAG pipeline.
